In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
import os

%matplotlib inline

In [3]:
# konfigurasi
video_path = 'D:\\Tim Lalu Lintas\\TA-Lalu-Lintas\\vidio\\TimeVideo_20250123_092456.mp4'

#direktori untuk dataset YOLO
output_yolo_images = 'dataset_yolo/images/'
output_yolo_labels = 'dataset_yolo/labels/'
os.makedirs(output_yolo_images,exist_ok = True)
os.makedirs(output_yolo_labels,exist_ok=True)

#direktori untuk dataset CNN
output_cnn_notUsing = 'dataset_cnn/notUsingHelm/'
output_cnn_usingHelm = 'dataset_cnn/usingHelm/'
os.makedirs(output_cnn_notUsing, exist_ok = True)
os.makedirs(output_cnn_usingHelm,exist_ok = True)

In [4]:
# variable global
drawing = False
ix,iy = -1,-1
boxes = []
current_class = 0 # 0 

untuk motor,1 untuk helm
frame_count = 0
paused = False
fps = 25 #frame persecond


In [5]:
# function drawing
def draw_box(event,x,y,flags,param):
    global ix , iy, drawing, img, boxes, current_class

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix,iy = x,y
    
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = img.copy()
            if current_class == 0:
                color = (0,255,0)
            else:
                color = (255,0,0)
            cv2.rectangle(img_copy,(ix,iy),(x,y),color,2)
            cv2.imshow('Frame',img_copy)
    
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        if current_class == 0:
            color = (0,255,0)
        else:
            color = (255,0,0)
        
        cv2.rectangle(img,(ix,iy),(x,y),color,2)
        x_min,y_min = min(ix,x), min(iy,y)
        x_max,y_max = max(ix,x), max(iy,y)
        boxes.append((current_class,x_min,y_min,x_max,y_max))
        cv2.imshow('Frame',img)

In [ ]:
# load video
cap = cv2.VideoCapture(video_path)
cv2.namedWindow('Frame')
cv2.setMouseCallback('Frame',draw_box)

while cap.isOpened():
    if not paused:
        time.sleep(1/fps)
        ret,frame = cap.read()
        if not ret:
            print("Video Selesai!")
            break
        img = frame.copy()
        cv2.imshow('Frame', img)
    
    key = cv2.waitKey(1)

    if key == 32: # spasi untuk play/pause
        paused = not paused
    
    elif key == ord('m'): # class untuk motor
        current_class = 0

    elif key == ord('h'): # class untuk helm
        current_class = 1
    
    elif key == ord('r'):
        img = frame.copy()
        boxes.clear()

    elif key == ord('n'): # Lanjut tanpa menyimpan frame
        paused = False
        boxes.clear()
    
    elif key == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        exit()

    elif key == 13:
        if len(boxes) == 0:
            print('Tidak ada label, frame tidak disimpan')
        else:
            #Simpan label dalam format yolo
            label_file = os.path.join(output_yolo_labels,f'frame_{frame_count:04d}.txt')
            with open(label_file,'w') as f:
                motor_boxes = []
                helmet_boxes = []

                # memisahkan motor dan helm
                for class_id, x_min, y_min, x_max, y_max in boxes:
                    x_center = (x_min + x_max) / 2 / frame.shape[1]
                    y_center = (y_min + y_max) / 2 / frame.shape[0]
                    width = (x_max - x_min) / frame.shape[1]
                    height = (y_max - y_min) / frame.shape[0]
                    f.write(f'{class_id} {x_center} {y_center} {width} {height}\n')

                    if class_id == 0:
                        motor_boxes.append((x_min,y_min,x_max,y_max))
                    elif class_id == 1:
                        helmet_boxes.append((x_min,y_min,x_max,y_max))

            #simpan gambar Data
            image_file = os.path.join(output_yolo_images, f'frame_{frame_count:04d}.jpg')
            cv2.imwrite(image_file, frame)

            # cut head untuk dataset klasifikasi CNN
            for motor_box in motor_boxes:
                x_min,y_min,x_max,y_max = motor_box

                # Pastikan koordinat dalam batas gambar
                y_crop_min = max(y_min, 0)
                y_crop_max = min(y_min + int(0.3 * (y_max - y_min)), frame.shape[0])
                
                if y_crop_max <= y_crop_min:  # Jika crop tidak valid
                    print(f"Head crop kosong pada frame {frame_count}, koordinat tidak valid!")
                    print(y_crop_max,y_crop_min)
                    continue  # Skip iterasi ini

                head_crop = frame[y_crop_min:y_crop_max, x_min:x_max]

                #memeriksa apakah helm ada di dalam bounding box motor
                helmet_found = False
                for helmet_box in helmet_boxes:
                    hx_min,hy_min,hx_max,hy_max = helmet_box
                    #jika helm berada diarea motor
                    if hx_min > x_min and hy_min > y_min and hx_max < x_max and hy_max < y_max:
                        helmet_found = True
                        break 

                if frame is None:
                    print("Frame kosong! Tidak bisa menyimpan gambar.")
                if head_crop is None or head_crop.size == 0:
                    print(f"Head crop kosong pada frame {frame_count}, tidak disimpan.")
                else:
                    if helmet_found:
                        cv2.imwrite(os.path.join(output_cnn_usingHelm, f'head_{frame_count:04d}.jpg'), head_crop)
                    else:
                        cv2.imwrite(os.path.join(output_cnn_notUsing, f'head_{frame_count:04d}.jpg'), head_crop)
                     
            
            frame_count += 1
            paused = False  

    
 
cap.release()
cv2.destroyAllWindows()


: 